# Clean dropbox and mongodb

We have a lot of tests and sometimes simply forget to clean the dropbox. This is a simple code to clean the dropbox.

In [3]:
import sys

import dropbox
from dropbox.exceptions import AuthError
from decouple import config

In [5]:
app_key = config("APP_KEY")
app_secret = config("APP_SECRET")
refresh_token = config("REFRESH_TOKEN")

clean the configs

In [6]:
def clean_dummies_from_folder(folder_path:str):
    folder_path = folder_path.strip("/")

    folder_path = "/" + folder_path + "/"
    with dropbox.Dropbox(
        app_key=app_key,
        app_secret=app_secret,
        oauth2_refresh_token=refresh_token,
    ) as dbx:
        # Check that the access token is valid
        try:
            dbx.users_get_current_account()
        except AuthError:
            sys.exit("ERROR: Invalid access token.")

        folders_results = dbx.files_list_folder(path=folder_path)
        entries = folders_results.entries
        for entry in entries:
            if "dummy" in entry.name:
                print("Deleting folder: " + entry.name)
                full_path = folder_path + entry.name

                print("Deleting folder: " + full_path)
                _ = dbx.files_delete_v2(path=full_path)

In [14]:
backend_config_path = "/Backend_files/Config/"
clean_dummies_from_folder(backend_config_path)


Deleting folder: dummy24f0f
Deleting folder: /Backend_files/Config/dummy24f0f


Clean the finished jobs

In [15]:
folder_path = "/Backend_files/Finished_Jobs/"
clean_dummies_from_folder(folder_path)

Deleting folder: dummy24f0f
Deleting folder: /Backend_files/Finished_Jobs/dummy24f0f
Deleting folder: dummy4f2ab
Deleting folder: /Backend_files/Finished_Jobs/dummy4f2ab


In [16]:
folder_path = "/Backend_files/Queued_Jobs/"
clean_dummies_from_folder(folder_path)

Deleting folder: dummy24f0f
Deleting folder: /Backend_files/Queued_Jobs/dummy24f0f


In [17]:
folder_path = "/Backend_files/Result/"
clean_dummies_from_folder(folder_path)

Deleting folder: dummy24f0f
Deleting folder: /Backend_files/Result/dummy24f0f


In [18]:
folder_path = "/Backend_files/Status/"
clean_dummies_from_folder(folder_path)

Deleting folder: dummy24f0f
Deleting folder: /Backend_files/Status/dummy24f0f


## Mongodb

For the mongodb we have to do similiar things.

In [1]:
from pymongo.mongo_client import MongoClient
from bson.objectid import ObjectId
from bson.errors import InvalidId

In [4]:
mongodb_username = config("MONGODB_USERNAME")
mongodb_password = config("MONGODB_PASSWORD")
mongodb_database_url = config("MONGODB_DATABASE_URL")
uri = f"mongodb+srv://{mongodb_username}:{mongodb_password}@{mongodb_database_url}"
uri = uri + "/?retryWrites=true&w=majority"

# Create a new client and connect to the server
client: MongoClient = MongoClient(uri)

In [6]:
database = client["backends"]
collection = database["configs"]
# get all the documents in the collection configs and save the disply_name in a list

backend_names: list = []
for config_dict in collection.find():
    if "display_name" in config_dict:
        if "dummy" in config_dict["display_name"]:
            print("Deleting config: " + config_dict["display_name"])
            collection.delete_one({"_id": ObjectId(config_dict["_id"])})
    if "payload" in config_dict:
        if "display_name" in config_dict["payload"]:
            print(config_dict["payload"])
            if "dummy" in config_dict["payload"]["display_name"]:
                print("Deleting config: " + config_dict["payload"]["display_name"])
                collection.delete_one({"_id": ObjectId(config_dict["_id"])})
        else:
            print("Deleting config: " + config_dict["display_name"])
            collection.delete_one({"_id": ObjectId(config_dict["_id"])})
    else:
        print("Deleting random config")
        collection.delete_one({"_id": ObjectId(config_dict["_id"])})
        

{'description': 'Setup of a cold atomic gas experiment with a single qudit.', 'version': '0.3', 'display_name': 'singlequdit', 'cold_atom_type': 'spin', 'gates': [{'coupling_map': [[0]], 'description': 'Evolution under Lx', 'name': 'rlx', 'parameters': ['omega'], 'qasm_def': 'gate lrx(omega) {}'}, {'coupling_map': [[0]], 'description': 'Evolution under the Z gate', 'name': 'rlz', 'parameters': ['delta'], 'qasm_def': 'gate rlz(delta) {}'}, {'coupling_map': [[0]], 'description': 'Evolution under lz2', 'name': 'rlz2', 'parameters': ['chi'], 'qasm_def': 'gate rlz2(chi) {}'}, {'coupling_map': [[0]], 'description': 'Apply the full time evolution on the array.', 'name': 'sq_full', 'parameters': ['omega, delta, chi'], 'qasm_def': 'gate sq_full(omega, delta, chi) {}'}], 'max_experiments': 1000, 'max_shots': 1000000, 'simulator': True, 'supported_instructions': ['rlx', 'rlz', 'rlz2', 'barrier', 'measure', 'load', 'sq_full'], 'num_wires': 1, 'wire_order': 'interleaved', 'num_species': 1, 'pending

Now also clean out stupid collections of finished jobs.

In [7]:
database = client["jobs"]

# now get a list of all existing collections within the database
collections = database.list_collection_names()
print(collections)
for name in collections:
    if "dummy" in name:
        print("Deleting collection: " + name)
        collection = database[name]
        collection.drop()

['finished.dummya8b41', 'finished.dummy92554', 'finished.dummydcc97', 'finished.dummy2e429', 'finished.dummy064de', 'finished.dummyd9eea', 'finished.dummy0e7ac', 'finished.dummy41ed5', 'finished.dummydbfa0', 'finished.dummy05cc0', 'finished.dummy60cf0', 'finished.dummyce76d', 'finished.dummy5b006', 'finished.dummye3211', 'finished.dummy2a94a', 'finished.dummy24c33', 'finished.dummy4f02c', 'finished.dummy8bd5d', 'finished.dummy1b479', 'finished.dummyc6d9c', 'finished.dummy4fdab', 'finished.dummy9dc94', 'finished.dummyacaf1', 'finished.dummy254e2', 'finished.dummya9f0d', 'finished.dummybcb4d', 'finished.dummyf7ae7', 'finished.dummy6f4e9', 'finished.dummyb4c4d', 'finished.dummy5edaf', 'finished.dummydf1d0', 'finished.dummy22039', 'finished.dummy46e11', 'finished.dummy53bef', 'finished.dummy557ee', 'finished.dummyb22d1', 'finished.dummy7fcff', 'finished.dummy64f5a', 'finished.dummyefa0c', 'finished.dummyb07c0', 'finished.dummy7d7a3', 'finished.dummy2dcbc', 'finished.dummybcecb', 'finished.